In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
pd.options.display.max_colwidth = 500
from xgboost import XGBClassifier

In [2]:
basedir = "C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/oaei/OAEI_w2v_steps_walklength1_2019_06_17_19_25_17_912781/"

gs = pd.read_csv(basedir+"oaei_gold_standard5best.csv", encoding="UTF-8", sep="\t", header=None)
gs.columns = ['src_id','tgt_id']
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['tgt_id'], right_on=['label'])
gs.head()

FileNotFoundError: File b'C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/oaei/OAEI_w2v_steps_walklength1_2019_06_17_19_25_17_912781/oaei_gold_standard5best.csv' does not exist

In [320]:
oaei_gold_standard3 = pd.read_csv(basedir+"oaei_gold_standard2.csv_merged.csv",sep="\t",encoding="UTF-8")
oaei_gold_standard3 = oaei_gold_standard3[['src_id','tgt_id','label']]
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
oaei_gold_standard3 = oaei_gold_standard3.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
oaei_gold_standard3 = oaei_gold_standard3.merge(embs, left_on=['tgt_id'], right_on=['label'])

oaei_gold_standard3 = extend_features(oaei_gold_standard3)
oaei_gold_standard3['syntactic_diff'] = oaei_gold_standard3.apply(lambda row: jacc(row['src_id'], row['tgt_id']), axis=1)
oaei_gold_standard3['plus_diff'] = oaei_gold_standard3.apply(lambda row: lev(row['src_id'], row['tgt_id']), axis=1)
oaei_gold_standard3 = oaei_gold_standard3.drop(['label'], axis=1)
oaei_gold_standard3.rename({'label_x':'label'}, inplace=True, axis='columns')
oaei_gold_standard3 = oaei_gold_standard3.drop(['label_y'], axis=1)
oaei_gold_standard3.head()

.
.


,src_id,tgt_id,label,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_18,tgt_19,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,syntactic_diff,plus_diff
0,http://dbkwik.webdatacommons.org/darkscape/resource/rune_plateskirt_(t),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/mithril_plateskirt_(t),0,0.317491,-0.027560,-0.621030,-1.268277,-0.792701,0.341311,0.158709,...,1.148980,0.287945,0.059706,0.975842,0.914466,3.468142,3.487075,1.201870,0.32000,0.259259
1,http://dbkwik.webdatacommons.org/darkscape/resource/mithril_plateskirt,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/mithril_plateskirt_(t),0,0.426119,0.340412,-0.196404,-1.426936,-0.330230,0.243810,-0.056750,...,1.148980,0.287945,0.074936,0.973114,0.914466,3.462294,3.487075,1.345385,0.24000,0.148148
2,http://dbkwik.webdatacommons.org/darkscape/resource/black_plateskirt_(t),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/mithril_plateskirt_(t),0,0.265712,0.215524,0.132091,-1.089106,-0.599757,0.418338,0.028266,...,1.148980,0.287945,0.056315,0.842697,0.914466,3.203301,3.487075,1.156991,0.32000,0.259259
3,http://dbkwik.webdatacommons.org/darkscape/resource/adamant_plateskirt_(t),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/mithril_plateskirt_(t),0,0.250850,0.083627,-0.014822,-0.894471,-0.459247,0.542112,0.062672,...,1.148980,0.287945,0.053087,0.862233,0.914466,2.981992,3.487075,1.165825,0.32000,0.259259
4,http://dbkwik.webdatacommons.org/darkscape/resource/rune_plateskirt_(t),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/black_plateskirt_(t),0,0.317491,-0.027560,-0.621030,-1.268277,-0.792701,0.341311,0.158709,...,1.420711,0.303725,0.071591,0.975842,0.847851,3.468142,3.601320,1.343899,0.26087,0.200000


In [321]:
origindir = "C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/data/oaei_data/"
labels1 = dict()
categories1 = dict()
with open(origindir+"graph_triples_darkscape.nt", encoding="UTF-8", mode="r") as f:
    for line in f:
        if '<http://www.w3.org/2000/01/rdf-schema#label>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/2000/01/rdf-schema#label ")
            labels1[line[0]] = line[1]
        if '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/1999/02/22-rdf-syntax-ns#type ")
            if line[1] not in ['http://www.w3.org/2002/07/owl#class','http://www.w3.org/1999/02/22-rdf-syntax-ns#property']:
                categories1[line[0]] = 'resource'
            else:
                categories1[line[0]] = line[1]
labels2 = dict()
categories2 = dict()
with open(origindir+"graph_triples_oldschoolrunescape.nt", encoding="UTF-8", mode="r") as f:
    for line in f:
        if '<http://www.w3.org/2000/01/rdf-schema#label>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/2000/01/rdf-schema#label ")
            labels2[line[0]] = line[1]
        if '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/1999/02/22-rdf-syntax-ns#type ")
            if line[1] not in ['http://www.w3.org/2002/07/owl#class','http://www.w3.org/1999/02/22-rdf-syntax-ns#property']:
                categories2[line[0]] = 'resource'
            else:
                categories2[line[0]] = line[1]

In [288]:
gs.loc[:,'src_category'] = 'resource'
gs.loc[:,'tgt_category'] = 'resource'
for index, row in gs.iterrows():
    try:
        gs.loc[index, 'src_category'] = categories1[row['src_id']]
    except KeyError:
        pass
    try:
        gs.loc[index, 'tgt_category'] = categories2[row['tgt_id']]
    except KeyError:
        pass
gs = gs.loc[gs.src_category == gs.tgt_category]
len(gs)

31537

In [308]:
from sklearn.metrics.pairwise import *
def extend_features(df):
    src_pattern = "src_\d+"
    tgt_pattern = "tgt_\d+"
    src_dim = int(len([elem for elem in [re.match(src_pattern, elem) is not None for elem in df.columns.values.tolist()] if elem==True]))
    tgt_dim = int(len([elem for elem in [re.match(tgt_pattern, elem) is not None for elem in df.columns.values.tolist()] if elem==True]))


    def dotproduct(v1, v2):
        result = list()
        for i in range(len(v1)):
            result.append([np.dot(v1[i], v2[i])])
        return np.array(result)

    def length(v):
        return np.sqrt(dotproduct(v, v))

    def angle(v1, v2):
        return np.arctan(dotproduct(v1, v2) / (length(v1) * length(v2)))

    a = np.array(df[["src_" + str(i) for i in range(src_dim)]].values.tolist())
    b = np.array(df[["tgt_" + str(i) for i in range(tgt_dim)]].values.tolist())
    print(".")
    df['src_tgt_angle'] = paired_cosine_distances(a, b)
    print(".")
    src_origin = np.full((len(df), src_dim), 0.0000001)
    tgt_origin = np.full((len(df), tgt_dim), 0.0000001)
    df['src_angle_to_origin'] = paired_cosine_distances(tgt_origin,a)
    #print(".")
    df['tgt_angle_to_origin'] = paired_cosine_distances(src_origin,b)
    df['src_veclen'] = length(a)
    df['tgt_veclen'] = length(b)
    df['src_tgt_veclen'] = paired_euclidean_distances(a,b)#.diagonal()#length(a-b)
    df.head()
    
    df.fillna(0, inplace = True)
    return df


In [322]:
gs = extend_features(gs)
oaei_gold_standard3 = extend_features(oaei_gold_standard3)
len(gs)

.
.
.
.


13892

In [310]:
memo = {}

def jacc(s,t, n=3):
    s = labels1[s]
    t = labels2[t]
    t = set([t[i:i+n] for i in range(len(t)-n+1)])
    s = set([s[i:i+n] for i in range(len(s)-n+1)])
    return 1-len([gram for gram in s if gram in t])/max(len(s), len(t))

def lev(s,t, n=3):
    s = labels1[s]
    t = labels2[t]
    return levenshtein(s,t)/max(len(s),len(t))
    
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1

    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])

    return res
gs['syntactic_diff'] = gs.apply(lambda row: jacc(row['src_id'], row['tgt_id']), axis=1)
gs['plus_diff'] = gs.apply(lambda row: lev(row['src_id'], row['tgt_id']), axis=1)

In [311]:
gs.loc[gs.src_tgt_angle<0]

,src_id,tgt_id,prediction,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_19,label_y,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,syntactic_diff,plus_diff


In [312]:
#for index, row in gs.iterrows():
#get_training_material(
nid="http://dbkwik.webdatacommons.org/darkscape/resource/ancient_magicks"
possibles = gs.loc[gs.src_id==nid]
possibles.sort_values(by='src_tgt_angle', ascending=False).head()

,src_id,tgt_id,prediction,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_19,label_y,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,syntactic_diff,plus_diff
1549,http://dbkwik.webdatacommons.org/darkscape/resource/ancient_magicks,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magikcks,0,0.036952,0.413745,-1.145034,-0.779795,-0.561282,0.82278,0.698887,...,0.242527,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magikcks,0.508886,0.818682,0.972749,3.447171,0.700109,3.162694,0.157895,0.047619
1548,http://dbkwik.webdatacommons.org/darkscape/resource/ancient_magicks,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magiks,0,0.036952,0.413745,-1.145034,-0.779795,-0.561282,0.82278,0.698887,...,0.266565,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magiks,0.490019,0.818682,0.965567,3.447171,0.687532,3.152513,0.166667,0.050000
1543,http://dbkwik.webdatacommons.org/darkscape/resource/ancient_magicks,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magics,0,0.036952,0.413745,-1.145034,-0.779795,-0.561282,0.82278,0.698887,...,0.262544,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magics,0.469559,0.818682,1.024296,3.447171,0.714869,3.127255,0.166667,0.050000
1545,http://dbkwik.webdatacommons.org/darkscape/resource/ancient_magicks,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magicks_tablet,0,0.036952,0.413745,-1.145034,-0.779795,-0.561282,0.82278,0.698887,...,0.073240,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magicks_tablet,0.460298,0.818682,0.973244,3.447171,3.199371,3.196007,0.360000,0.259259
1546,http://dbkwik.webdatacommons.org/darkscape/resource/ancient_magicks,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magicks_tablet,0,0.036952,0.413745,-1.145034,-0.779795,-0.561282,0.82278,0.698887,...,0.073240,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magicks_tablet,0.460298,0.818682,0.973244,3.447171,3.199371,3.196007,0.360000,0.259259


In [296]:
coefs = clf.coef_
for c in range(len(coefs[0])):
    print(cols[c] + " -> " + str(coefs[0,c]))

src_0 -> -0.05573210302852501
src_1 -> 0.2792166415931157
src_2 -> 0.2661866042448061
src_3 -> 0.06901310175878989
src_4 -> -0.09214621888925652
src_5 -> -0.07979407271356746
src_6 -> -0.052418195675345565
src_7 -> 0.08803693259867118
src_8 -> 0.06211771626764272
src_9 -> -0.12100768826171404
src_10 -> -0.06611730510112615
src_11 -> -0.05307372010339453
src_12 -> -0.2053213574224535
src_13 -> -0.11359506065677277
src_14 -> 0.4181218598281776
src_15 -> 0.13059880297797094
src_16 -> 0.28965914969221657
src_17 -> -0.21147142501957933
src_18 -> -0.21747900189987893
src_19 -> -0.22508516894716155
tgt_0 -> 0.09499593925259897
tgt_1 -> -0.212277769930554
tgt_2 -> -0.021534368207363448
tgt_3 -> 0.0415538652191134
tgt_4 -> -0.048406010316357745
tgt_5 -> -0.12302124790630388
tgt_6 -> 0.13605745521576657
tgt_7 -> -0.5537647959263519
tgt_8 -> -0.3327497203671292
tgt_9 -> 0.15203643425351387
tgt_10 -> 0.7236883178033088
tgt_11 -> -0.2140580377921243
tgt_12 -> 0.15291176600481765
tgt_13 -> 0.0489911

In [313]:
gs.to_csv('C:/Users/D072202/Desktop/oaei_gs.csv', encoding="UTF-8",sep="\t")

In [319]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
#oaei_gold_standard3 = pd.read_csv(basedir+"oaei_gold_standard2.csv_merged.csv",sep="\t",encoding="UTF-8")

cols = [col for col in oaei_gold_standard3.columns if col not in ['plus_diff','syntactic_diff','label','Unnamed: 0','src_id','tgt_id','src_category','tgt_category']]#['src_tgt_angle', 'src_tgt_veclen', 'plus_diff', 'syntactic_diff']
X, y = oaei_gold_standard3[cols], oaei_gold_standard3.label
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.9,0:0.1}).fit(X, y)
#XGBClassifier().fit(X, y)
#LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.9,0:0.1}).fit(X, y)
        #random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.1,0:0.9}).fit(X, y)

X = gs[cols]
matchings = gs.loc[clf.predict(X)==1]
matchings.loc[:,'confidence'] = clf.predict_proba(matchings[cols])[:,1]
len(matchings)

AttributeError: 'DataFrame' object has no attribute 'label'

In [78]:
matchings = gs.loc[gs.plus_diff<0.25]

In [259]:
gs = matchings
len(matchings)

13887

In [260]:
len(set(gs.src_id))
from gensim.models import Doc2Vec, Word2Vec
model = Word2Vec.load(basedir+"w2v.model")
def get_training_material(nid):
            res = list()
            with open(basedir+"w2v_training_material.csv", mode="r", encoding="UTF-8") as f:
                for line in f:
                    if nodeid in line.split(" "):
                        for w in line.split(" "):
                            yield w

def mergedf(df1, df2):
            if df1 is None:
                return df2
            else:
                return df1.append(df2, ignore_index=True)


In [326]:
gs.loc[:,'total_score'] = 0

In [327]:
progress = 0
matchings = None
total = len(set(gs.src_id))
for nodeid in set(gs.src_id):#.union(gs.tgt_id)
                possible_matches_for_nodeid = gs.loc[(gs.src_id==nodeid) ]
        
                #possible_matches.loc[((possible_matches.src_id==nodeid) & (possible_matches.tgt_id.isin(get_possible_matches(nodeid))))]



                progress += 1
                if len(possible_matches_for_nodeid)<1:
                    continue
                elif len(possible_matches_for_nodeid) == 1:
                    matching_pair = possible_matches_for_nodeid.head(1)
                    matchings = mergedf(matchings, matching_pair)
                    continue
                
                
                #print(str(progress), end="\r")
                print("         Computing rank-features: " + str(int(100*progress/total)) + "%.", end='\r')
                # In[312]:



                #model.docvecs.most_similar(0)


                # In[313]:


                #print('Closest in general:')
                #for val in model.docvecs.most_similar(i):
                #    try:
                #        print(documents_ids_A[int(val[0])])
                #    except:
                #        try:
                #            print(documents_ids_B[int(val[0])])
                #        except:
                #            print(str(val[0]) + " not found")


                # In[314]:


                ##print('Closest in terms of cosine similarity:')
                ##vecs = model.docvecs.doctag_syn0[np.array(get_possible_matches(nodeid))]
                ##vecs = model.wv[get_possible_matches(nodeid)]
                ##x = cosine_similarity(np.array([model.wv[nodeid]]), vecs)
                ##x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                ##sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=False)
                ##sorted_x.loc[:,'cos_score'] = 0
                #ctr = 1
                ##sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
                ##sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
                ##sorted_x['cos_sim'] = sorted_x['cos_sim'].astype('float64')
                sorted_x = possible_matches_for_nodeid.sort_values(by=['src_tgt_angle'], ascending=True)
                sorted_x.loc[:,'cos_score'] = 0
                #maximum = sorted_x.head(1).src_tgt_angle.values[0]
                #sorted_x.loc[:,'diff_to_max'] = 1.0 - sorted_x.loc[:, 'src_tgt_angle'] / maximum
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['cos_sim']))
                    sorted_x.loc[index, 'cos_score'] = row['cos_score'] + 1/ctr
                    ctr += 1
                #sorted_x = possible_matches_for_nodeid.sort_values(by=['confidence'], ascending=False)
                sorted_x.loc[:,'conficence'] = 0 
                sorted_x.loc[:,'confidence_score'] = 0 
                ctr = 1
                for index, row in sorted_x.iterrows():
                    sorted_x.loc[index, 'confidence_score'] = row['confidence_score'] + 1/ctr
                    ctr += 1

                # In[315]:


                ##print('Closest in terms of Euclidean distance:')print('Closest in terms of Euclidean distance:')
                #sorted_x2 = sorted_x
                ##vecs = model.wv[get_possible_matches(nodeid)]
                ##x = euclidean_distances(np.array([model.wv[nodeid]]), vecs)
                ##x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                ##sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
                sorted_x = sorted_x.sort_values(by=['src_tgt_veclen'], ascending=True)
                sorted_x.loc[:,'euclid_score'] = 0
                ctr = 1
                ##sorted_x.columns = ['euclid_sim' if col==0 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['euclid_sim']))
                    sorted_x.loc[index, 'euclid_score'] = row['euclid_score'] + 1/ctr
                    ctr += 1



                #print('Closest in terms of syntax:')
                sorted_x2 = sorted_x
                #vecs = model.wv[get_possible_matches(nodeid)]
                def edits(v1, v2s):
                    res = list()
                    v1 = v1.split("/")[-1]
                    for v2 in v2s:
                        v2 = v2.split("/")[-1]
                        res.append(editdistance.eval(v1, v2)/min(len(v1), len(v2)))
                    return np.array([res])
                #x = edits(nodeid, get_possible_matches(nodeid))
                #x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                #sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
                sorted_x = possible_matches_for_nodeid.sort_values(by=['syntactic_diff'], ascending=True)
                sorted_x.loc[:,'syntax_score'] = 0
                ctr = 1
                sorted_x.columns = ['syntax_diff' if col==0 else col for col in sorted_x.columns]
                #for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['syntax_diff']))
                #    sorted_x.loc[index, 'syntax_score'] = row['syntax_score'] + 1/ctr
                #    ctr += 1



                sorted_x3 = sorted_x
                #o1 = get_training_material(nodeid)
                #inverted_dict = dict()
                #for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
                #    inverted_dict[tgt_id] = [tgt_id]
                #    for word in get_training_material(tgt_id):
                #        if word in inverted_dict.keys():
                #            inverted_dict[word].append(tgt_id)
                #        else:
                #            inverted_dict[word] = [tgt_id]

                #tgt_scores = dict()
                #for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
                #    tgt_scores[tgt_id]=0

                #for tupl in model.predict_output_word(o1, topn=99999999999999):
                #    if tupl[0] in inverted_dict.keys():
                #        for tgt_id in inverted_dict[tupl[0]]:
                #            tgt_scores[tgt_id] = tgt_scores[tgt_id] + tupl[1]
                sorted_x.loc[:,'probability'] = 0
                sorted_x.loc[:,'probability_score'] = 0
                #ctr=1
                #for item in sorted(tgt_scores.items(), key = lambda kv:(kv[1], kv[0]), reverse=True):
                #    sorted_x.loc[sorted_x.tgt_id==item[0],'probability_score'] = 1/ctr
                #    sorted_x.loc[sorted_x.tgt_id==item[0],'probability'] = float(item[1])
                #    ctr += 1

                # In[316]:


                #print('Closest in sum:')
                #sorted_x[['probability_score','probability']].merge(..., left_index=True, right_index=True)
                x = sorted_x[['probability_score','probability']].merge(sorted_x3['syntax_score'].to_frame().merge(sorted_x2, left_index=True, right_index=True), left_index=True, right_index=True)
                x.loc[:,'total_score'] = x['syntax_score'] + x['euclid_score'] + x['probability_score'] + x['confidence_score'] + x['cos_score']
                sorted_x = x.sort_values(by=['total_score'], ascending=False)
                ##sorted_x.columns = ['tgt_id' if col==1 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():#sorted_x.loc[sorted_x.total_score == max(sorted_x.total_score.values),:].iterrows():
                    matching_pair = pd.DataFrame([sorted_x.loc[index]])
                    matching_pair.loc[:,'src_id'] = nodeid
                    #print(nodeid + "\t" + row[1] + "\t" + str(row['total_score']) + "\t" + str(row['cos_score']) + "\t" + str(row['euclid_score']))
                    matchings = mergedf(matchings, matching_pair)
                

KeyboardInterrupt: 

In [263]:
#matchings = matchings_saved
#matchings = gs

In [264]:
matchings_saved=matchings
matchings = matchings.sort_values(by=['total_score','src_tgt_veclen'], ascending=[False, True])
married_matchings = None
ctr = 0
while len(matchings) > 0:
                ctr += 1
                row = matchings.head(1)
                married_matchings = mergedf(married_matchings, pd.DataFrame(row))
                matchings = matchings.loc[~(matchings.src_id == row.src_id.values[0]) & ~(matchings.tgt_id == row.tgt_id.values[0])]
                print(str(len(matchings)) + " left     ", end="\r")

In [78]:
matchings.loc[matchings.tgt_id=='http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/ancient_magicks']

,syntax_score,src_id,tgt_id,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,src_category,tgt_category,src_tgt_angle,src_veclen,tgt_veclen,src_tgt_veclen,syntactic_diff,plus_diff,euclid_score,total_score


In [ ]:
gold = pd.read_csv(origindir+"gold_standard.csv",encoding="UTF-8",sep="\t", header=None)
relevants = set(gold[0].to_list()+gold[1].to_list())
married_matches = None
for index, row in married_matchings.iterrows():
        if row['src_id'] in relevants and row['tgt_id'] in relevants:
            married_matches = mergedf(married_matches, pd.DataFrame(row).transpose())
married_matches.head()

In [265]:
import os
def create_elem(src_id, tgt_id):
    elem = '<map>\n<Cell>\n<entity1 rdf:resource="'+src_id+'"/>\n'
    elem = elem + '<entity2 rdf:resource="'+tgt_id+'"/>\n<relation>=</relation>\n'
    elem = elem + '<measure rdf:datatype="xsd:float">1.0</measure>\n</Cell>\n</map>'
    return elem

matchings_filename ="PROBmarried_matchings.csv"
#married_matches = pd.read_csv(basedir + matchings_filename, sep="\t", encoding="UTF-8")
starttag = '<?xml version="1.0" encoding="utf-8"?>\n<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<Alignment>\n  <xml>yes</xml>\n  <level>0</level>\n  <type>??</type>\n  <onto1>\n    <Ontology rdf:about="darkscape">\n      <location>http://darkscape.wikia.com</location>\n    </Ontology>\n  </onto1>\n  <onto2>\n    <Ontology rdf:about="oldschoolrunescape">\n      <location>http://oldschoolrunescape.wikia.com</location>\n    </Ontology>\n  </onto2>\n'
endtag = '</Alignment>\n</rdf:RDF>'
#os.mkdir(basedir + matchings_filename.replace(".csv",""))
with open(basedir + matchings_filename.replace(".csv","") + str(os.sep) + 'darkscape~oldschoolrunescape~results.xml', "w+", encoding="UTF-8") as f:
        f.write(starttag)
        for index, row in married_matchings.iterrows():
            f.write(create_elem(str(row.src_id).replace("&","&amp;"), str(row.tgt_id).replace("&","&amp;"))+"\n")
        f.write(endtag)



In [143]:
import os
def create_elem(src_id, tgt_id):
    elem = '<map>\n<Cell>\n<entity1 rdf:resource="'+src_id+'"/>\n'
    elem = elem + '<entity2 rdf:resource="'+tgt_id+'"/>\n<relation>=</relation>\n'
    elem = elem + '<measure rdf:datatype="xsd:float">1.0</measure>\n</Cell>\n</map>'
    return elem

matchings_filename ="onlyembedding_ML.csv"
married_matches = pd.read_csv("C:/Users/D072202/RData2Graph/rdata2graph/data/sap_hilti_data/oaei_gold_standard1best.csv", sep="\t", encoding="UTF-8")
married_matches.columns=['src_id','tgt_id','label']
starttag = '<?xml version="1.0" encoding="utf-8"?>\n<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<Alignment>\n  <xml>yes</xml>\n  <level>0</level>\n  <type>??</type>\n  <onto1>\n    <Ontology rdf:about="darkscape">\n      <location>http://darkscape.wikia.com</location>\n    </Ontology>\n  </onto1>\n  <onto2>\n    <Ontology rdf:about="oldschoolrunescape">\n      <location>http://oldschoolrunescape.wikia.com</location>\n    </Ontology>\n  </onto2>\n'
endtag = '</Alignment>\n</rdf:RDF>'
#os.mkdir(basedir + matchings_filename.replace(".csv",""))
with open(basedir + matchings_filename.replace(".csv","") + str(os.sep) + 'darkscape~oldschoolrunescape~results.xml', "w+", encoding="UTF-8") as f:
        f.write(starttag)
        for index, row in matchings.iterrows():
            f.write(create_elem(str(row.src_id).replace("&","&amp;"), str(row.tgt_id).replace("&","&amp;"))+"\n")
        f.write(endtag)

